## 1.2.6 例:素数判定

ここでは、整数$n$の素数性をチェックする2つの方法を紹介する。  
- 増加オーダーが$\Theta(\sqrt{n})$であるアルゴリズム  
- 増加オーダーが$\Theta(\log{n})$である"確率的"なアルゴリズム  

#### 約数を探す
ある数値が素数であるかテストする⽅法のひとつとして、  
その数値の約数を探すというものがある。  
次のプログラムは、$2$から始まる⼀連の数字$n$について割り切れるかどうかを調べるという素直な⽅法で、約数を探している。

In [1]:
(define (smallest-divisor n)
  (find-divisor n 2)
  )
(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))
    )
  )
(define (divides? a b)
  (= (remainder b a) 0)
  )
(define (square x) (* x x))

動作イメージ  
例:23

    (smallest-divisor 23)
    (find-divisor 23 2)    ;2^2 < 23
    (find-divisor 23 3)    ;3^2 < 23
    (find-divisor 23 4)    ;4^2 < 23
    (find-divisor 23 5)    ;5^2 > 23
    23  

例:25

    (smallest-divisor 25)
    (find-divisor 25 2)    ;2^2 < 25
    (find-divisor 25 3)    ;3^2 < 25
    (find-divisor 25 4)    ;4^2 < 25
    (find-divisor 25 5)    ;5^2 = 25
    5

数値が素数であるかは次のようにテストできる:$n$は、$n$⾃⾝がその最⼩の約数である場合、かつその場合に限り、素数である。

In [2]:
(define (prime? n)
  (= n (smallest-divisor n))
  )
; 動作確認
(display (prime? 27))
(newline)
(display (prime? 109))
(newline)

False
True


find-divisorの終了条件は、もし$n$が素数でないならば、  
それは$\sqrt{n}$以下の約数を持つという事実に基づいている。  
つまり、このアルゴリズムは、$1$から$\sqrt{n}$までの約数についてだけテストすればよい。よって、このプログラムの必要なステップ数の増加オーダーは$\Theta(\sqrt{n})$となる。

#### フェルマーテスト(確率的アルゴリズム)
フェルマーの小定理(整数論の結果)に基づいたステップ数の増加オーダーが$\Theta(\log{n})$であるアルゴリズムを紹介する。  


##### フェルマーの小定理
$n$が素数で、$a$が$n$より⼩さい任意の正の整数であるとき、  
$a$の$n$乗は法$n$に関して$a$と合同である。

つまり、  
素数$n$に対して,$n>\forall{a}>0$ $\Rightarrow$ $mod(a,n) \equiv mod(a^n,n).$  
待遇をとると、  
$\exists a(n>a>0), mod(a,n) \not\equiv mod(a^n,n)$ $\Rightarrow$ $n$は素数でない.  
また、すべての$a$に対して$mod(a,n) \equiv mod(a^n,n)$が成り立ったとしても$n$は素数であるとは限らないので注意すること。   
このような整数$n$をカーマイケル数という。

In [3]:
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp)(remainder (square (expmod base (/ exp 2) m)) m))
        (else (remainder (* base (expmod base (- exp 1) m)) m)))
  )

In [4]:
(define (fermat-test n)
  (define (try-it a)
    (= (expmod a n n) a))
  (try-it (+ 1 (random (- n 1))))
  )

In [5]:
(define (fast-prime? n times)
  (cond ((= times 0) True)
        ((fermat-test n) (fast-prime? n (- times 1)))
        (else False)
         )
  )

In [6]:
(display (fast-prime? 1039 10))

True

動作イメージ  
(remainder手続きをmod()で表記)  
例:base=3 exp=5 m=5(n=5,a=3としてnが素数であるかのテスト)

$mod(3,5)=3$  
$mod(3^5,5)=mod(243,5)=3$  
$expmod(3,5,5)$  
$=mod(3\cdot expmod(3,4,5),5)$  
$=mod(3\cdot mod(expmod(3,2,5)^2,5),5)$  
$=mod(3\cdot mod(mod(expmod(3,1,5)^2,5)^2,5),5)$  
$=mod(3\cdot mod(mod(mod(3\cdot expmod(3,0,5),5)^2,5)^2,5),5)$  
$=mod(3\cdot mod(mod(mod(3\cdot 1,5)^2,5)^2,5),5)$  
$=mod(3\cdot mod(mod(mod(3,5)^2,5)^2,5),5)$  
$=mod(3\cdot mod(mod(3^2,5)^2,5),5)$  
$=mod(3\cdot mod(mod(9,5)^2,5),5)$  
$=mod(3\cdot mod(4^2,5),5)$  
$=mod(3\cdot mod(16,5),5)$  
$=mod(3\cdot 1,5)$  
$=mod(3,5)$  
$=3$  



例:base=5 exp=6 m=6(n=6,a=5としてnが素数であるかのテスト)

$mod(5,6)=5$  
$mod(5^6,6)=mod(15625,6)=1$  
$expmod(5,6,6)$  
$=mod(expmod(5,3,5)^2,6)$  
$=mod(mod(5\cdot expmod(5,2,5),6)^2,6)$  
$=mod(mod(5\cdot mod(expmod(5,1,6)^2,6),6)^2,6)$  
$=mod(mod(5\cdot mod(mod(5 \cdot expmod(5,0,6),6)^2,6),6)^2,6)$  
$=mod(mod(5\cdot mod(mod(5 \cdot 1,6)^2,6),6)^2,6)$  
$=mod(mod(5\cdot mod(mod(5,6)^2,6),6)^2,6)$  
$=mod(mod(5\cdot mod(5^2,6),6)^2,6)$  
$=mod(mod(5\cdot mod(25,6),6)^2,6)$  
$=mod(mod(5\cdot 1,6)^2,6)$  
$=mod(mod(5,6)^2,6)$  
$=mod(1^2,6)$  
$=mod(1,6)$  
$=1$  


##### 練習問題
- [練習問題1.21 smallest-divisor手続きの確認](../exercises/1.21.ipynb)
- [練習問題1.22 search-for-primes](../exercises/1.22.ipynb)
- [練習問題1.23 素数判定で使用するnext手続き](../exercises/1.23.ipynb)
- [練習問題1.24 timed-prime-test手続きの実行時間](../exercises/1.24.ipynb)
- [練習問題1.25 expmod](../exercises/1.25.ipynb)
- [練習問題1.26 実行速度が遅いfast-prime?](../exercises/1.26.ipynb)
- [練習問題1.27 素数判定・カーマイケル数](../exercises/1.27.ipynb)
- [練習問題1.28 素数判定・ミラー-ラビンテスト](../exercises/1.28.ipynb)